# Senkrechter Wurf mit Luftwiderstand

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Bewegungsgleichung für den senkrechten Wurf mit Luftwiderstand auf verschiedene Arten numerisch gelöst. Dazu werden das Euler-Verfahren und das modifizierte Euler-Verfahren implementiert. Da die analytische Lösung bekannt ist, kann der Fehler der numerischen Lösung untersucht werden. Abschließend wird eine Funktion aus der SciPy-Bibliothek zur Lösung der Differentialgleichung herangezogen.

## Euler-Verfahren

### Bewegungsgleichung

Die Bewegungsgleichung für den senkrechten Wurf lautet in dimensionsloser Form

$$\frac{\text{d}v}{\text{d} t} = -1 - |v|v \,.$$

Die folgende Funktion berechnet die Beschleunigung $\text{d}v/\text{d}t$ als Funktion der Geschwindigkeit $v$. Sie wird in allen Teilen dieses Jupyter-Notebooks verwendet.

In [ ]:
def dv_dt(v):
    return -1-abs(v)*v

### Berechnung der Lösung der Differentialgleichung

Die nächste Funktion berechnet die Lösung der Bewegungsgleichung mit Hilfe des Euler-Verfahrens unter Verwendung der gerade definierten Funktion `dv_dt`. Die Zeitentwicklung beginnt bei $t=0$ mit der Anfangsgeschwindigkeit `v_0` und endet bei `t_end`. Dabei werden insgesamt `n_steps` Iterationsschritte ausgeführt. 

Zunächst wird aus der Endzeit `t_end` und der Zahl der Iterationsschritte `n_steps` die Größe eines Zeitschritts berechnet. Anschließend werden zwei Listen `t_values` und `v_values` erzeugt und mit den entsprechenden Anfangswerten vorbelegt. In der Schleife wird dann jeweils ein Iterationsschritt des Euler-Verfahrens ausgeführt und die neuen Werte für Zeit und Geschwindigkeit an die Listen angehängt.

Nach Abschluss der Berechnung werden die Listen mit den Zeitpunkten und den Geschwindigkeiten zurückgegeben.

In [ ]:
def euler(t_end, n_steps, v_0):
    delta_t = t_end/n_steps
    t_values = [0]
    v_values = [v_0]
    for n in range(n_steps):
        t_values.append((n+1)*delta_t)
        v = v_values[-1]
        v_values.append(v + dv_dt(v)*delta_t)
    return t_values, v_values

### Importanweisungen

Für die graphische Benutzerschnittstelle benutzen wir das `ipywidgets`-Modul, insbesondere unter Verwendung des `interact`-Dekorators. Zur graphischen Darstellung der Ergebnisse wird die `matplotlib`-Bibliothek verwendet. Außerdem werden in der Datei `numphyspy.style` einige Eigenschaften der graphischen Darstellung eingestellt.

In diesem Juypter-Notebook führen wird die Imports erst aus, wenn sie wirklich benötigt werden. In allen folgenden Jupyter-Notebooks finden sich die Import-Anweisungen gleich zu Beginn, da sie normalerweise am Beginn eines Python-Programms stehen sollten. Die Anweisungen in der folgenden Zelle werden immer wieder in den Jupyter-Notebooks vorkommen und dann nicht mehr gesondert erklärt werden.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern können die Werte der folgenden Variablen eingestellt werden:
- `t_end`: Endzeitpunkt
- `n_steps`: Zahl der Iterationsschritte
- `v_0`: Anfangsgeschwindigkeit

Der Dekorator `@interact` sorgt dafür, dass bei Veränderungen der Werte die Funktion `plot_result` aufgerufen wird, die die Berechnung durchführt und das Ergebnis graphisch darstellt.

Falls gewünscht, können die vorgegebenen Grenzen oder Schrittweiten für die Regler durch Anpassung der Werte in `widget_dict` modifiziert werden.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=5, min=1, max=20, step=1,
                   description=r"$t_\text{end}$"),
               "n_steps":
               widgets.IntSlider(
                   value=500, min=10, max=5000, step=10,
                   description=r"$n_\text{steps}$"),
               "v_0":
               widgets.FloatSlider(
                   value=4, min=-5, max=5, step=0.1,
                   description="$v(0)$")
               }

@interact(**widget_dict)
def plot_euler_result(t_end, n_steps, v_0):
    t_values, v_values = euler(t_end, n_steps, v_0)

    fig, ax = plt.subplots()
    ax.plot(t_values, v_values)
    ax.set_xlabel("$t$")
    ax.set_ylabel("$v$")

## Fehleranalyse für das Euler-Verfahren

### Importanweisungen

Für die Berechnung der exakten Lösung der Bewegungsgleichung des senkrechten Wurfs mit Luftwiderstand benötigen wir einige Funktionen aus dem `math`-Modul der Standardbibliothek, nämlich den Tangens, den Tangens Hyperbolicus und die zugehörigen inversen Funktionen.

In [ ]:
from math import atan, atanh, tan, tanh

### Exakte Lösung

Nun werden Funktionen zur Berechnung der exakten Lösung der Bewegungsgleichung des senkrechten Wurfs mit Luftwiderstand definiert. Wie im Buch erklärt, muss man hier zwischen drei Bereichen für die Anfangsgeschwindigkeit `v_0` unterscheiden:
1. positive Anfangsgeschwindigkeiten
1. negative Anfangsgeschwindigkeiten, die größer als die asymptotische Geschwindigkeit sind
1. negative Anfangsgeschwindigkeiten, die kleiner als die asymptotische Geschwindigkeit sind.

Zunächst werden Funktionen für diese drei Fälle definiert. Die vierte Funktion ist dafür zuständig, in Abhängigkeit von `v_0` aus den ersten drei Funktionen die passende auszuwählen. Anschließend werden für die gewünschten Zeitwerte die zugehörigen Geschwindigkeiten berechnet und in einer Liste zurückgegeben.

In [ ]:
def v_exact_pos(t, v_0):
    t_0 = atan(v_0)
    if t < t_0:
        return tan(t_0-t)
    else:
        return -tanh(t-t_0)

def v_exact_neg1(t, v_0):
    return -tanh(t+atanh(-v_0))

def v_exact_neg2(t, v_0):
    return -1/tanh(t+atanh(-1/v_0))

def v_exact(t_values, v_0):
    v_values = []
    if v_0 > 0:
        v = v_exact_pos
    elif v_0 > -1:
        v = v_exact_neg1
    else:
        v = v_exact_neg2
    for t in t_values:
        v_values.append(v(t, v_0))
    return v_values

### Berechnung des absoluten Fehlers

Zunächst wird hier die numerische Lösung der Bewegungsgleichung bestimmt, wobei die Funktion `method` die bereits definierte Funktion `euler` für das Euler-Verfahren oder die weiter unten definierte Funktion `mod_euler` für das modifizierte Euler-Verfahren sein kann. Bei Bedarf können auch noch weitere Funktionen definiert werden, um deren Fehler zu analysieren. Diese Funktionen müssen zwei Listen mit den Zeiten und den zugehörigen Geschwindigkeiten zurückgeben.

Anschließend wird die exakte Lösung für die Anfangsgeschwindigkeit `v_0` sowie die Zeitwerte `t_values`, für die auch die numerische Lösung vorliegt, berechnet.

In der folgenden Schleife, die parallel die beiden Listen mit den numerischen und exakten Werten abarbeitet, wird dann der absolute Fehler bestimmt und in der Liste `error_values` gesammelt.

In [ ]:
def abs_errors(t_end, n_steps, v_0, method):
    t_values, v_numerical_values = method(
        t_end, n_steps, v_0)
    v_exact_values = v_exact(t_values, v_0)
    error_values = []
    for v1, v2 in zip(v_numerical_values, v_exact_values):
        error_values.append(abs(v1-v2))
    return t_values, error_values

### Graphische Darstellung des absoluten Fehlers

Aus dem oben definierten Dictionary `widget_dict` wird der Eintrag `n_steps` wiederverwendet. Dies hat zur Folge, dass sich eine Änderung des Werts von `n_steps` auch in der Benutzerschnittstelle für das Euler-Verfahren auswirkt und die Lösung dort neu berechnet wird. Eine solche Kopplung werden wir in späteren Jupyter-Notebooks vermeiden, da solche Kopplungen unter Umständen unnötigerweise zeitaufwändige Rechnungen anstoßen.

In [ ]:
@interact(**widget_dict)
def plot_error_results(n_steps):
    v_0 = 4
    t_end = 4
    t_values, error_values = abs_errors(t_end, n_steps,
                                        v_0, euler)

    fig, ax = plt.subplots()
    ax.plot(t_values, error_values, linestyle="None",
            marker="o")
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\delta v$")

## Modifiziertes Euler-Verfahren

### Berechnung der Lösung mit Hilfe des modifizierten Euler-Verfahrens

Die Funktion `mod_euler` unterscheidet sich von der Funktion `euler` lediglich im Iterationsschritt zur Berechnung der nächsten Geschwindigkeit. Hierbei werden die Größen `k_1` und `k_2` berechnet, wobei `k_2` gleich der Differenz zwischen der alten und der neuen Geschwindigkeit ist.

In [ ]:
def mod_euler(t_end, n_steps, v_0):
    delta_t = t_end/n_steps
    t_values = [0]
    v_values = [v_0]
    for n in range(n_steps):
        t_values.append((n+1)*delta_t)
        v = v_values[-1]
        k_1 = dv_dt(v)*delta_t
        k_2 = dv_dt(v+k_1/2)*delta_t
        v_values.append(v + k_2)
    return t_values, v_values

### Vergleich der absoluten Fehler des Euler-Verfahrens und des modifizierten Euler-Verfahrens

Hier werden `n_out` verschiedene Iterationsschrittanzahlen betrachtet, die auf einer logarithmischen Skala äquidistant sind. Anschließend wird für die beiden oben definierten Funktionen `euler` und `mod_euler` mit Hilfe der bereits definierten Funktion `abs_errors` der jeweilige absolute Fehler berechnet und in einer Liste aufgesammelt. Das Ergebnis `error_values` wird zusammen mit der entsprechenden Liste der Zeitschritte zurückgegeben.

In [ ]:
def error_comparison(t_end, n_steps_min, n_steps_max,
                     n_out, v_0):
    error_values = [[], []]
    dt_values = []
    factor = n_steps_max/n_steps_min
    for m in range(n_out):
        n_steps = int(n_steps_min * factor**(m/(n_out-1)))
        dt_values.append(t_end / n_steps)
        for idx, method in enumerate((euler, mod_euler)):
            _, errors = abs_errors(t_end, n_steps,
                                   v_0, method)
            error_values[idx].append(errors[-1])
    return dt_values, error_values

### Darstellung der Resultate

Hier verzichten wir auf eine graphische Benutzerschnittstelle, sondern zeigen das Vorgehen, wenn man die Parameter direkt in der Code-Zelle festlegt. Bei Bedarf können die Werte in der Zelle angepasst werden. Die ursprüngliche Parametereinstellung sieht eine Anfangsgeschwindigkeit `v_0` von 4 vor und betrachtet den absoluten Fehler bei der Zeit `t_end` gleich 0,2 in der Nähe des Nulldurchgangs der Geschwindigkeit. Es werden 20 verschiedene Werte für die Anzahl der Iterationsschritte zwischen $10$ und $10^7$ betrachtet. 

Man sieht, dass das modifizierte Euler-Verfahren deutlich schneller konvergiert. Für kleine Zeitschritte wirken sich jedoch Rundungsfehler aus.

*Hinweis:* Die Auswertung dieser Zelle kann ein klein wenig länger dauern.

In [ ]:
t_end = 0.2
n_steps_min = 10
n_steps_max = 1e7
n_out = 20
v_0 = 4

dt_values, error_values = error_comparison(
    t_end, n_steps_min, n_steps_max, n_out, v_0)

fig, ax = plt.subplots()
ax.loglog(dt_values, error_values[0], label="Euler",
          linestyle="None", marker="o")
ax.loglog(dt_values, error_values[1], label="mod. Euler",
          linestyle="None", marker="o")
ax.set_xlabel(r"$\Delta t$")
ax.set_ylabel(r"$\delta v$")
ax.legend(loc="lower right");

## Integration unter Verwendung von SciPy

### Importanweisungen

Nun greifen wir zur Lösung der Differentialgleichung auf die numerischen Programmbibliotheken NumPy und SciPy zurück. Die folgenden `import`-Anweisungen entsprechen den üblichen Konventionen. Für das Paket `numpy` wird die Abkürzung `np` eingeführt. Da das `scipy`-Paket aus zahlreichen Unterpaketen besteht, wird lediglich der Namensraum des hier benötigten Unterpakets `integrate` importiert.

In [ ]:
import numpy as np
from scipy import integrate

### Anpassung der Zeitableitung der Geschwindigkeit für die Benutzung mit SciPy

Die Zeitableitung der Geschwindigkeit hängt zwar nicht explizit von der Zeit ab, aber SciPy ist für den allgemeineren Fall vorbereitet und verlangt daher die Zeit `t` als erstes Argument. Die Funktion `dv_dt_scipy` gibt die Aufgabe einfach an die oben definierte Funktion `dv_dt` weiter.

In [ ]:
def dv_dt_scipy(t, v):
    return dv_dt(v)

### Integration mit der SciPy-Funktion `integrate.solve_ivp`

Die bisherigen Parameter werden hier um Fehlerschranken für den absoluten Fehler (`atol`) und den relativen Fehler (`rtol`) ergänzt. Zunächst wird ein Array erzeugt, das die Zeitwerte enthält, zu denen die Lösung berechnet werden soll. Anschließend erfolgt die Lösung mit Hilfe der SciPy-Funktion `integrate.solve_ivp`, die standardmäßig ein Runge-Kutta-Verfahren der Ordnung 5(4) verwendet. Nachdem die Lösung der Differentialgleichung berechnet ist, wird der absolute Fehler am Ende des berechneten Zeitintervalls bestimmt. Das Objekt `solution` enthält eine ganze Reihe von Informationen, von denen wir die folgenden verwenden werden:
- `solution.t`: Zeitpunkte, zu denen die Lösung vorliegt
- `solution.y`: Lösung zu den gerade genannten Zeitpunkten
- `solution.nfev`: Anzahl der Aufrufe von `dv_dt_scipy` und damit von `dv_dt`

Die Bedeutung weiterer Bestandteile von `solution` können der [Dokumentation von `solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) entnommen werden.

In [1]:
def scipy_integration(t_end, n_steps, v_0, atol, rtol):
    t_values = np.linspace(0, t_end, n_steps+1)
    solution = integrate.solve_ivp(
        dv_dt_scipy, (0, t_end), (v_0,),
        t_eval=t_values, atol=atol, rtol=rtol)
    error = abs(solution.y[0, -1]
                - v_exact((t_end,), v_0)[0])
    return solution, error

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Zunächst wird die graphische Benutzerschnittstelle um Schieberegler für die Schranken `atol` und `rtol` für den absoluten bzw. relativen Fehler ergänzt. Nach der Berechnung der Lösung der Differentialgleichung werden die Anzahl der Funktionsaufrufe von `dv_dt` und der Fehler am Ende des Zeitintervalls ausgegeben. Abschließend wird die Zeitabhängigkeit der Geschwindigkeit dargestellt, die dem eingangs gefundenen Ergebnis entsprechen sollte. Allerdings wird das Ergebnis hier mit erheblich weniger Rechenaufwand erhalten, wie man durch Vergleich der Zahl der Aufrufe von `dv_dt` mit der Anzahl der Iterationsschritte in den vorigen Rechnungen erkennt.

In [ ]:
new_widgets = {"atol":
               widgets.FloatLogSlider(
                   value=1e-9, min=-12, max=-2, step=0.1,
                   description="atol"),
               "rtol":
               widgets.FloatLogSlider(
                   value=1e-9, min=-12, max=-2, step=0.1,
                   description="rtol")
               }
widget_dict.update(new_widgets)

@interact(**widget_dict)
def plot_scipy_results(t_end, n_steps, v_0, atol, rtol):
    solution, error = scipy_integration(t_end, n_steps,
                                        v_0, atol, rtol)
    print(f"{solution.nfev} Aufrufe der Funktion dv_dt")
    print(f"absoluter Fehler: {error:8.2e}")
    print()

    fig, ax = plt.subplots()
    ax.plot(solution.t, solution.y[0, :])
    ax.set_xlabel("$t$")
    ax.set_ylabel("$v$")